In [1]:
def countSentence(s):
    sentences = sent_tokenize(s)
    return len(sentences)  
def removeTitleNumber(line):
    foundEnglish = False
    newline = ''
    words = line.split()
    for w in words:
        if re.search(r'[A-Za-z]', w[0]):
            foundEnglish = True

        if foundEnglish == False:
            continue
        else:
            newline = newline + ' ' + w
    newline = newline.strip()
    return newline

def removeTitleNumber2(line):
    foundEnglish = False
    output = ''
    words = line.split()
    for w in words:
        if foundEnglish:
            output += ' ' + w
        else:
            if re.search(r'[A-Za-z]', w[0]):
                foundEnglish = True
                output += ' ' + w
            if isSpecialChar(w[0]) == True:
                output += ' ' + w
            else:
                continue
    output = output.strip()
    return output

########
def checkSentenceEnd(s):
    # check sentence is finished or not
    if s[-1] in ['.',',',';',':']:
        return True
    else:
        return False
def checkLowercase(s):
    if re.search(r'[a-z]', s) is None:
        return False
    else:
        return True
def checkUppercase(s):
    if re.search(r'[A-Z]',s) is None:
        return False
    else:
        return True
# For PDF
def isTitle(words):
    for word in words:
        if word in ['to','for','toward','by','on','a','an']:
            continue
        if re.search(r'[A-Z]',word[0]) is None:
            return False
    return True
def checkPdfFormatError(s):
    words = word_tokenize(s)
    if len(words) <= 5:
        return False,'LengthError'
    elif s.count('..') > 0:
        return False,'DuplicatePeriodError'
    else:
        return True,'Pass'
def countSentence(s):
    # break paragraph into sentences
    sentences = sent_tokenize(s)
    # if contains two or more sentences, the paragraph is valid
    return len(sentences)

def isSpecialChar(char):
    bulletSymbol = [61623,8226]
    if ord(char) in bulletSymbol :
        return True
    else:
        return False

In [5]:
import io
import os
import boto3
import json
# import hashlib
# import requests
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
# from elasticsearch import Elasticsearch
# from elasticsearch.helpers import bulk 
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import logging
from docx import Document
import collections
import sys

# initialize for reading document
resourceManager = PDFResourceManager()
retStr = io.StringIO()
laParams = LAParams()
converter = TextConverter(resourceManager, retStr, codec='utf-8', laparams=laParams)
interpreter = PDFPageInterpreter(resourceManager, converter)
count = 0

# use for store all paragraphs
allParas = []
# use for store valid paragraphs
validPara = []
# buffer status variable
PrevSentIsFinished = True
# temp variable
temp = ''
# add to temp count
tempCount = 0
# add to temp max retries
countThreshold = 3
# allow sentence count as a paragraph
sentenceThreshold = 1
# valid sentences list
validSentence = []

filePath = 'mobile-policy.pdf'

# Read all para and store to allParas list
with open(filePath, "rb") as file:
    # read pages
    for pageNumber, page in enumerate(PDFPage.get_pages(file)):
            if pageNumber > 2:
                break
            interpreter.process_page(page)
            content = retStr.getvalue()

            # read para
            paras = content.split('\n\n') 
            for para in paras:
                # store to allParas list
                allParas.append(para)

            retStr.truncate(0)
            retStr.seek(0)

    # Find out duplicated sentences
    duplicatedParas = []
    for item, count in collections.Counter(allParas).items():
        if count > 1: 
            duplicatedParas.append(item)

# Read again, and avoid duplicate sentences, then store valid sentences to validParas list
with open(filePath, "rb") as file:
    # read pages
    for pageNumber, page in enumerate(PDFPage.get_pages(file)):
            #print("========== %d =========="  %pageNumber)
            if pageNumber <= 1:
                continue
            interpreter.process_page(page)
            content = retStr.getvalue()

            # read para
            paras = content.split('\n\n') #sent_tokenize(content)
            for para in paras:
                # remove duplicated lines
                if para in duplicatedParas:
                    continue

                # remove spaces from start and end
                para = para.strip()
                # remove inner spaces
                para = ' '.join(para.split())
                # remove special chars
                para = removeTitleNumber(para)
                # append to validPara list
                validPara.append([para,pageNumber+1])
            retStr.truncate(0)
            retStr.seek(0)

#Start processing validPara list
for paraList in validPara:
    para = paraList[0]
    pageNumber = paraList[1]
    if para == '':
        continue
    if countSentence(para) >= sentenceThreshold:
        # split sentences from paragraph
        sentenceList = sent_tokenize(para)
        for sentence in sentenceList:
            # buffer status not finished
            if PrevSentIsFinished == False:
                # if current sentence has uppercase at the first letter, give up temp
                if checkUppercase(sentence[0]):
                    # set buffer status to finished
                    PrevSentIsFinished = True
                    # clear buffer
                    temp = ''
                    # initiate count
                    tempCount = 0
                # setup max retries to prevent keep adding sentences to buffer with no ending
                elif tempCount > countThreshold:
                    # set buffer status to finished
                    PrevSentIsFinished = True
                    # clear buffer
                    temp = ''
                    # initiate count
                    tempCount = 0
                else:
                    # add current sentence to buffer
                    temp = temp + sentence
                    # pull buffer to sentence
                    sentence = temp

            
            # Check is title or not
            words = word_tokenize(sentence)
            if isTitle(words):
                validSentence.append( ["TITLE",pageNumber] )
            
            # 
            if isSpecialChar(sentence[0]):
                bulletSymbol = sentence[0]
                bulletSentenceList = sentence.split(bulletSymbol)
                for bulletSentence in bulletSentenceList:
                    if bulletSentence == '':
                        continue
                    validSentence.append([bulletSymbol + bulletSentence,pageNumber])
            # Check sentence prefix capitalization
            if checkUppercase(sentence[0]):
                # Check sentence contain lowercase letters
                if checkLowercase(sentence):
                    # Check sentence end
                    isFinished = checkSentenceEnd(sentence)
                    if isFinished:
                        # Check sentence format
                        isValid, errorMsg = checkPdfFormatError(sentence)
                        if isValid:
                            # add valid sentence to validSentence list 
                            validSentence.append([sentence,pageNumber])
                        else:
                            #logger.info("(" + errorMsg +'): ' + sentence)
                            continue

                        # set buffer status to finished
                        PrevSentIsFinished = True
                        # clear temp
                        temp = ''
                        # initiate count
                        tempCount = 0
                    # if sentence not finished, add sentence to buffer, change buffer status to unfinished
                    else: 
                        # push buffer
                        temp = temp + sentence + ' '
                        # add count
                        tempCount += 1
                        # set buffer status to unfinished
                        PrevSentIsFinished = False

In [6]:
# count sentence in validSentence list
sent_count = 0
for sentenceList in validSentence:
    sentence = sentenceList[0]
    pageNumber = sentenceList[1]
    
    # if sentence is title, pass
    if sentence == 'TITLE':
        sent_count += 1
        continue
    # First sentence in the list
    if sent_count == 0:
        prev_sent = None
        next_sent = validSentence[sent_count+1][0]
    # Last sentence in the list
    elif sent_count == len(validSentence)-1:
        prev_sent = validSentence[sent_count-1][0]
        next_sent = None
    # Other sentences in the list
    else:
        prev_sent = validSentence[sent_count-1][0]
        next_sent = validSentence[sent_count+1][0]
    # if pre and next sentence is title, modify it to None
    if prev_sent == 'TITLE':
        prev_sent = None
    if next_sent == 'TITLE':
        next_sent = None
    # print logger
    print('---')
    #print("(Prev) ",prev_sent)
    #print("(Curr) ",sentence)
    #print("(Next) ",next_sent)
    print(sentence)
    # add index count
    sent_count += 1

---
All mobile devices attempting to connect to the District network through an unmanaged network (i.e.
---
Devices that have not been previously approved by IT, are not in compliance with IT’s security policies, or represent any threat to the District network or data will not be allowed to connect.
---
Laptop computers or personal PCs may only access the District network using a Virtual Private Network (VPN) connection.
---
Employees using mobile devices and related software for network and data access will, without exception, use secure data management procedures.
---
All mobile devices must be protected by a strong password.
---
See the WCCCD’s password policy for additional details.
---
Employees agree to never disclose their passwords to anyone.
---
All users of mobile devices must employ reasonable physical security measures.
---
End users are expected to secure all such devices used for this activity whether or not they are actually in use and/or being carried.
---
This includes